# Claim Feature Modelling

This notebook looks at some object models for patent claim features.

We have two main types of patent claim: apparatus and method.  

The "apparatus" claim type includes system and kit claims. This claim covers a physical object or set of objects.

The "method" claim type covers a method or process. It consists of a sequence of steps.

A physical object has multiple parts. These may feature in the "apparatus" patent claim. The object or its parts may have limitations that define the structure of the object or part. An object or its parts are defined by nouns, whereas the interaction between parts is defined by verbs (commonly in the past tense). We can refer to an object, an object part, or a system or set of objects as an "entity".

Steps of a "method" claim are verbs using the present participle.

Our "apparatus" claim may be represented as a series of nested graphs. 

A single graph may be represented as:
```
graph = { "a" : ["c"],
          "b" : ["c", "e"],
          "c" : ["a", "b", "d", "e"],
          "d" : ["c"],
          "e" : ["c", "b"],
          "f" : []
        }
```
We would have something similar, but each node would be an "entity" representation. 

In [1]:
c = [1, 2, 3]
c += [2]
print(c)
c.pop(2)
print(c)

[1, 2, 3, 2]
[1, 2, 2]


In [2]:
import spacy

nlp = spacy.load('en') 

# Python graph class tutorial- http://www.python-course.eu/graphs_python.php
class StringFeature:
    """ Abstract object for a phrase or word.
     
     Attributes:
        headpos - the head part of speech, e.g. noun or verb
        tokens - tuple of (string, POS_tag)        
    """
    def __init__(self, string):
        """ Initialise object."""
        self.doc = nlp(string)
    
    def __repr__(self):
        return self.doc
    
    @property
    def stringphrase(self):
        return self.__repr__()
    
class ClaimFeature(StringFeature):
    """ Abstract object for instantiating entities and steps. 
    
    Attributes:
        ref_num - int representing associated reference number (maybe a list?)
        parent (? - or get from navigating children)
        children
        limitations
        essential - T or F (optional = F)
        number - (default = 1, >1 = plurality) 
        order - if in a set of children where it comes in the claim
    
    """
    def __init__(self, string):
        """ Initialise object. """
        super().__init__(string) 
        self.children = list()
        self.limitations = list()
       
    def add_child(self, child):
        """ Add a child entity.
        
        child: an object of the same class
        """
        # if string convert to ClaimFeature
        if isinstance(child, str):
            child = type(self)(child)
        assert isinstance(child, type(self))
        self.children.append(child)
        return self.children
    
    def remove_child(self, child):
        """ Remove a child entity.
        
        child: a child object to remove
        """
        self.children.remove(child)
        return self.children
    
    def add_limitation(self, limitation):
        """ Add a limitation.
        
        limitation: a Limitation object
        """
        if isinstance(limitation, str):
            limitation = Limitation(limitation)
        assert isinstance(limitation, Limitation)
        self.limitations.append(limitation)
        return self.limitations
    
    def remove_limitation(self, limitation):
        """ Remove a limitation.
        
       limitation: a Limitation object
        """
        self.limitations.remove(limitation)
        return self.limitations
    
    def prettyprint(self, object_str_single, object_str_plural, tabs=0):
        """ Pretty print a representation of feature with
        children and limitations.
        
        object_str_single: string to call feature instantiation 
        object_str_plural: string to call feature instantiation (plural)
        """
        tabtext = "\t"*tabs
        print("{0}{1}: {2}\n".format(tabtext, object_str_single, self.__repr__()))
        
        if self.limitations:
            tabs = tabs + 1
            tabtext = "\t"*tabs
            print("{0}Limitations:\n".format(tabtext))
            for i, limitation in enumerate(self.limitations):
                print("\t{0}{1} - {2}\n".format(tabtext, i, limitation.__repr__()))
        
        if self.children:
            tabs = tabs + 1
            print("{0}Child {1}:\n".format(tabtext, object_str_plural))
            for i, child in enumerate(self.children):
                child.prettyprint(tabs=tabs)
    
    
class Entity(ClaimFeature):
    """ Object to represent an entity in a claim. 
    
    Attributes:
        see ClaimFeature
        headpos = headnoun
    
    """
    def prettyprint(self, tabs=0):
        super().prettyprint("Entity", "Entities", tabs)
        

class Step(ClaimFeature):
    """ Object to represent a step in a claim. 
    
    Attributes:
        see ClaimFeature
        headpos = headnoun
        
    """
    def prettyprint(self, tabs=0):
        super().prettyprint("Step", "Steps", tabs)
        
        
class Limitation(StringFeature):
    """ Object to represent a limitation applied to an entity or method step.
    
    Attributes:
        see StringFeature
    
    """
    pass

In [3]:
class Claim:
    """ Abstract object to represent a claim. 
    
    Attributes:
        features
    """
    def __init__(self, features=[]):
        """ Initialise object.
        
        features - list of instantiated ClaimFeature objects
        """
        self.features = features

class MethodClaim(Claim):
    """ Object to represent an apparatus claim. 
    
    Attributes:
        steps
    """
    def __init__(self, steps=[]):
        """ Initialise object.
        
        steps - list of Step objects
        """
        super().__init__(steps) 
        
    @property
    def steps(self):
        return self.features

class ApparatusClaim(Claim):
    """ Object to represent an apparatus claim. 
    
    Attributes:
        entities  
    """
    def __init__(self, entities=[]):
        """ Initialise object.
        
        entities - list of Entity objects
        """
        super().__init__(entities) 
    
    @property
    def entities(self):
        return self.features

In [61]:
a = ApparatusClaim()
e = Entity("an apparatus")

In [62]:
e

an apparatus

In [63]:
e.prettyprint()

Entity: an apparatus

Limitations:

Child Entities:



In [64]:
a.features = [e, e]

In [65]:
a.entities

[an apparatus, an apparatus]

We could do with the init and add limitation working with strings that get converted behind the scenes?  

But POS won't be accurate on string portions. But when we are drafting we will not have POS.

Add child could take a string and convert into an entity/step behind the scenes?

For claim printing we want to generate a string representation from the parts.

In [66]:
e.tokens

[('an', 'DT'), ('apparatus', 'NN')]

In [91]:
e = Entity("an apparatus")
e.add_child("a chamber")
e.add_child("a lid")
e.add_limitation("width greater than length")
e.prettyprint()

Entity: an apparatus

	Limitations:

		0 - width greater than length

	Child Entities:

		Entity: a chamber

		Entity: a lid



In [92]:
e.children[1].add_limitation("covering the chamber")

[covering the chamber]

In [93]:
e.prettyprint()

Entity: an apparatus

	Limitations:

		0 - width greater than length

	Child Entities:

		Entity: a chamber

		Entity: a lid

			Limitations:

				0 - covering the chamber



In [94]:
e.children[1].add_child("depression")
e.prettyprint()

Entity: an apparatus

	Limitations:

		0 - width greater than length

	Child Entities:

		Entity: a chamber

		Entity: a lid

			Limitations:

				0 - covering the chamber

			Child Entities:

				Entity: depression



We need a subtype of limitation that is a "relationship" with an existing entity (or a relative limitation), e.g. in the above example "covering the chamber" ought to refer to the chamber as an entity.  

Also how do you handle "is a X" style statements (as opposed to "has a" or "comprises a")?

Limitation is more a property.

In [95]:
m = Step("method of washing")
m.add_child("washing face")
m.add_child("washing hands")
m.children[0].add_limitation("with soap")
m.prettyprint()

Step: method of washing

Child Steps:

	Step: washing face

		Limitations:

			0 - with soap

	Step: washing hands



In [96]:
m.children[0].tokens

[('washing', 'VBG'), ('face', 'NN')]

In [97]:
from nltk.corpus import wordnet as wn

In [106]:
synsets = wn.synsets('washing', pos=wn.VERB)

for ss in synsets:
    print(ss.lemmas(), ss.definition())

[Lemma('wash.v.01.wash'), Lemma('wash.v.01.rinse')] clean with some chemical process
[Lemma('wash.v.02.wash'), Lemma('wash.v.02.lave')] cleanse (one's body) with soap and water
[Lemma('wash.v.03.wash'), Lemma('wash.v.03.launder')] cleanse with a cleaning agent, such as soap, and water
[Lemma('wash.v.04.wash')] move by or as if by water
[Lemma('wash.v.05.wash')] be capable of being washed
[Lemma('wash.v.06.wash')] admit to testing or proof
[Lemma('wash.v.07.wash')] separate dirt or gravel from (precious minerals)
[Lemma('wash.v.08.wash')] apply a thin coating of paint, metal, etc., to
[Lemma('wash.v.09.wash'), Lemma('wash.v.09.wash_out'), Lemma('wash.v.09.wash_off'), Lemma('wash.v.09.wash_away')] remove by the application of water or other liquid and soap or some other cleaning agent
[Lemma('wash.v.10.wash')] form by erosion
[Lemma('moisten.v.01.moisten'), Lemma('moisten.v.01.wash'), Lemma('moisten.v.01.dampen')] make moist
[Lemma('lave.v.01.lave'), Lemma('lave.v.01.lap'), Lemma('lave.v

In [108]:
synsets = wn.synsets('face', pos=wn.NOUN)

for ss in synsets:
    print(ss.lemmas(), ss.definition(), ss.hypernyms())

[Lemma('face.n.01.face'), Lemma('face.n.01.human_face')] the front of the human head from the forehead to the chin and ear to ear [Synset('external_body_part.n.01')]
[Lemma('expression.n.01.expression'), Lemma('expression.n.01.look'), Lemma('expression.n.01.aspect'), Lemma('expression.n.01.facial_expression'), Lemma('expression.n.01.face')] the feelings expressed on a person's face [Synset('countenance.n.01')]
[Lemma('face.n.03.face')] the general outward appearance of something [Synset('appearance.n.01')]
[Lemma('face.n.04.face')] the striking or working surface of an implement [Synset('surface.n.01')]
[Lemma('face.n.05.face')] a part of a person that is used to refer to a person [Synset('person.n.01')]
[Lemma('side.n.04.side'), Lemma('side.n.04.face')] a surface forming part of the outside of an object [Synset('surface.n.02')]
[Lemma('face.n.07.face')] the part of an animal corresponding to the human face [Synset('external_body_part.n.01')]
[Lemma('face.n.08.face')] the side upon whi

In [109]:
synsets = wn.synsets('chamber', pos=wn.NOUN)

for ss in synsets:
    print(ss, ss.lemmas(), ss.definition(), ss.hypernyms())

Synset('chamber.n.01') [Lemma('chamber.n.01.chamber')] a natural or artificial enclosed space [Synset('enclosure.n.01')]
Synset('chamber.n.02') [Lemma('chamber.n.02.chamber')] an enclosed volume in the body [Synset('cavity.n.04')]
Synset('chamber.n.03') [Lemma('chamber.n.03.chamber')] a room where a judge transacts business [Synset('room.n.01')]
Synset('chamber.n.04') [Lemma('chamber.n.04.chamber')] a deliberative or legislative or administrative or judicial assembly [Synset('assembly.n.04')]
Synset('bedroom.n.01') [Lemma('bedroom.n.01.bedroom'), Lemma('bedroom.n.01.sleeping_room'), Lemma('bedroom.n.01.sleeping_accommodation'), Lemma('bedroom.n.01.chamber'), Lemma('bedroom.n.01.bedchamber')] a room used primarily for sleeping [Synset('room.n.01')]


Can use hypernyms and lowest_common_hypernyms to suggest alternative words?

Ambuigity may be measured as a metric of the number of synsets?

Can use hyponyms and/or similarity measures to compare with terms in prior art citations.

Each synset contains one or more lemmas, which represent a specific sense of a specific word. - we want to be using lemmas in patent drafts.